In [1]:
import requests
import datetime
import urllib.request
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import locale

# **I. Données "Classements"**

## **I.A. Aquisition et chargement**

### **Génération d'une liste des dates et heures correspondant au nom de tous les fichiers classements**

In [2]:
# Format du lien de téléchargement : https://www.vendeeglobe.org/download-race-data/vendeeglobe_20210305_080000.xlsx
# Premier classement téléchargeable : 2020-11-08 à 14h00
# 2021-01-27 à 14h = dernier classement où aucun bateau n'est encore arrivé au port (à partir du classement suivant, le format change)
# Heures quotidiennes théoriques de publication des classements - utilisées pour nommer les fichiers xlsx téléchargeables : 4-8-11-14-17-21 UTC ou 5h,9h,12h,15h,18h,22h FR
# Remarque : ces heures théoriques de publication ne sont pas toujours respectées mais les noms de fichiers suivent ce standard même en cas de décalage d'une ou deux heures

hours=[datetime.time(x).strftime('%H%M%S') for x in [4,8,11,14,17,21]]

first_full_day=datetime.date(2020,11,9)
last_full_day=datetime.date(2021,1,26)
full_days=last_full_day-first_full_day
day=datetime.timedelta(days=1)
days=[(first_full_day + i*day).strftime('%Y%m%d') for i in range(0,full_days.days+1)]

days_hours=[f'{days[i]}_{hours[j]}.xlsx' for i in range(0,len(days)) for j in range(0,len(hours))]

first_day_hours=['20201108_140000.xlsx','20201108_170000.xlsx','20201108_210000.xlsx']
last_day_hours=['20210127_040000.xlsx','20210127_080000.xlsx','20210127_110000.xlsx','20210127_140000.xlsx']

days_hours=first_day_hours+days_hours+last_day_hours

### **Téléchargement des fichiers (403 fichiers .xlsx avant l'arrivée du premier skipper)**

In [3]:
local_path='data/'
link="https://www.vendeeglobe.org/download-race-data/vendeeglobe_"
# Les deux lignes ci-dessous ont été utilisées pour télécharger les fichiers Excel
# Elles sont commentées car il s'agit d'instructions coûteuse qu'il est inutile de reproduire une fois les fichiers disponibles en local
# for day_hour in days_hours:
#     local_filename, headers = urllib.request.urlretrieve(link+day_hour,filename=local_path+day_hour)


In [4]:
print(f'Au total, on charge {len(days_hours)} fichiers Excel, soir {len(days_hours)} classements')

Au total, on charge 481 fichiers Excel, soir 481 classements


#### **Insertion du contenu de tous les fichiers de classement dans un grand data_frame, en insérant le date-time de publication de chaque classement comme colonne supplémentaire**

In [5]:
df=pd.DataFrame()
for day_hour in days_hours:
    new_ranking_df=pd.read_excel("data/converted/"+day_hour+".xls",header=4,skiprows=range(38,42),usecols=range(1,21))
    datetime_value=pd.to_datetime(day_hour.strip('.xlsx'),format='%Y%m%d_%H%M%S')
    new_ranking_df['Ranking publication']=datetime_value
    df=df.append(new_ranking_df)

## **I.B. Préparation des données**

### **Premières observations préalables**

    Affichage des caractéristiques du dataframe

In [6]:
df.head(5)

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Heure FR\nHour FR,Latitude\nLatitude,Longitude\nLongitude,Cap\nHeading,Vitesse\nSpeed,VMG\nVMG,Distance\nDistance,...,Vitesse\nSpeed.1,VMG\nVMG.1,Distance\nDistance.1,Cap\nHeading.2,Vitesse\nSpeed.2,VMG\nVMG.2,Distance\nDistance.2,Unnamed: 19,Unnamed: 20,Ranking publication
0,1,\nFRA 18,Louis Burton\nBureau Vallée 2,15:30 FR\n,46°24.46'N,01°50.48'W,241°,17.7 kts,17.5 kts,0.3 nm,...,0.0 kts,0.0 kts,2788.0 nm,201°,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm,2020-11-08 14:00:00
1,2,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,15:31 FR\n1min,46°24.34'N,01°49.82'W,241°,11.1 kts,10.9 kts,0.4 nm,...,0.0 kts,0.0 kts,2787.9 nm,196°,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm,2020-11-08 14:00:00
2,3,\nFRA 8,Jérémie Beyou\nCharal,15:30 FR\n,46°24.91'N,01°49.99'W,244°,15.5 kts,15.5 kts,0.5 nm,...,0.0 kts,0.0 kts,2788.5 nm,199°,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm,2020-11-08 14:00:00
3,4,\nFRA 59,Thomas Ruyant\nLinkedOut,15:30 FR\n,46°24.71'N,01°49.68'W,244°,13.2 kts,13.1 kts,0.7 nm,...,0.0 kts,0.0 kts,2788.3 nm,196°,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm,2020-11-08 14:00:00
4,5,\nFRA 53,Maxime Sorel\nV And B Mayenne,15:30 FR\n,46°24.59'N,01°49.56'W,246°,10.9 kts,10.9 kts,0.2 nm,...,0.0 kts,0.0 kts,2788.1 nm,195°,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm,2020-11-08 14:00:00


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15873 entries, 0 to 32
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Unnamed: 1           15873 non-null  object        
 1   Unnamed: 2           15873 non-null  object        
 2   Unnamed: 3           15873 non-null  object        
 3   Heure FR
Hour FR     13671 non-null  object        
 4   Latitude
Latitude    13671 non-null  object        
 5   Longitude
Longitude  13671 non-null  object        
 6   Cap
Heading          13671 non-null  object        
 7   Vitesse
Speed        13671 non-null  object        
 8   VMG
VMG              13671 non-null  object        
 9   Distance
Distance    13671 non-null  object        
 10  Cap
Heading.1        13671 non-null  object        
 11  Vitesse
Speed.1      13671 non-null  object        
 12  VMG
VMG.1            13671 non-null  object        
 13  Distance
Distance.1  13671 non-nul

    Suppression des \n dans les cellules qui en contiennent pour ne pas qu'elles polluent l'analyse par la suite

In [8]:
df[df.columns[1]].replace('.*\n','',regex=True,inplace=True)
df[df.columns[3]].replace('\n','',regex=True,inplace=True)
df[df.columns[2]].replace('\n',' / ',regex=True,inplace=True)
df.head(2)

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Heure FR\nHour FR,Latitude\nLatitude,Longitude\nLongitude,Cap\nHeading,Vitesse\nSpeed,VMG\nVMG,Distance\nDistance,...,Vitesse\nSpeed.1,VMG\nVMG.1,Distance\nDistance.1,Cap\nHeading.2,Vitesse\nSpeed.2,VMG\nVMG.2,Distance\nDistance.2,Unnamed: 19,Unnamed: 20,Ranking publication
0,1,FRA 18,Louis Burton / Bureau Vallée 2,15:30 FR,46°24.46'N,01°50.48'W,241°,17.7 kts,17.5 kts,0.3 nm,...,0.0 kts,0.0 kts,2788.0 nm,201°,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm,2020-11-08 14:00:00
1,2,MON 10,Boris Herrmann / Seaexplorer - Yacht Club De M...,15:31 FR1min,46°24.34'N,01°49.82'W,241°,11.1 kts,10.9 kts,0.4 nm,...,0.0 kts,0.0 kts,2787.9 nm,196°,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm,2020-11-08 14:00:00


### **Mise au propre du nom des colonnes à l'aide d'un multiindex**

In [9]:
lev1=['Last signal']+['Identification']*3+['Last signal']*2+["Last 30min"]*4+["Since last report"]*4+["Last 24h"]*4+['Last signal']*2+['Identification']
lev2=['Rank','Sail N°','Skipper / Crew','Hour description','Latitude','Longitude','Heading (°)','Speed (kts)','VMG (kts)','Distance (nm)','Heading (°)','Speed (kts)','VMG (kts)','Distance (nm)','Heading (°)','Speed (kts)','VMG (kts)','Distance (nm)','DTF (nm)','DTL (nm)','Ranking publication']
list_tuples=list(zip(lev1,lev2))
col_multi_index=pd.MultiIndex.from_tuples(list_tuples,names=['Category','Field'])
df.set_axis(col_multi_index,axis=1,inplace=True)
df.head(2)

Category Last signal Identification  \
Field           Rank        Sail N°   
0                  1         FRA 18   
1                  2         MON 10   

Category                                                                      \
Field                                        Skipper / Crew Hour description   
0                            Louis Burton / Bureau Vallée 2         15:30 FR   
1         Boris Herrmann / Seaexplorer - Yacht Club De M...     15:31 FR1min   

Category Last signal              Last 30min                        \
Field       Latitude   Longitude Heading (°) Speed (kts) VMG (kts)   
0         46°24.46'N  01°50.48'W        241°    17.7 kts  17.5 kts   
1         46°24.34'N  01°49.82'W        241°    11.1 kts  10.9 kts   

Category                ... Since last report                          \
Field    Distance (nm)  ...       Speed (kts) VMG (kts) Distance (nm)   
0               0.3 nm  ...           0.0 kts   0.0 kts     2788.0 nm   
1               0.4 nm  ...           0.0 kts   0.0 kts     2787.9 nm   

Category    Last 24h                                     Last signal           \
Field    Heading (°) Speed (kts) VMG (kts) Distance (nm)    DTF (nm) DTL (nm)   
0               201°     0.3 kts   0.3 kts        6.1 nm  24293.9 nm   0.0 nm   
1               196°     0.3 kts   0.2 kts        6.0 nm  24294.2 nm   0.4 nm   

Category      Identification  
Field    Ranking publication  
0        2020-11-08 14:00:00  
1        2020-11-08 14:00:00  

[2 rows x 21 columns]

### **Extraction de l'information relative à la date et à l'heure**

    a) Compréhension de l'information fournie sur les dates et heures

In [10]:
df[('Identification','Hour description')].value_counts().head(30)

14:30 FR          2067
08:30 FR          2064
04:30 FR          2064
21:30 FR          2062
17:30 FR          2039
11:30 FR          2034
05:30 FR           198
22:30 FR           198
15:30 FR           183
12:30 FR           165
09:30 FR           164
18:30 FR           131
18:00 FR            66
09:00 FR            33
12:00 FR            33
15:00 FR            32
11:00 FR            32
17:00 FR            27
04:00 FR-30min      11
21:00 FR-30min       9
17:00 FR-30min       7
08:00 FR-30min       6
15:31 FR1min         6
11:00 FR-30min       6
14:00 FR-30min       5
16:30 FR-60min       4
15:28 FR-2min        3
07:00 FR-90min       3
15:29 FR-1min        2
10:30 FR-60min       2
Name: (Identification, Hour description), dtype: int64

Analyse de la colonne heure : 
- les heures sans rectificatif du type +/-Xmin semblent être les heures pour lesquelles la collecte d'infos et la réponse obtenue coïncident  
- Lorsqu'il y a un rectificatif du type +/-Xmin, c'est probablement qu'on a pas eu de réponse du bateau donc on prend le dernier signal qu'on a reçu et on indique le décalage par rapport au moment de tentative de collecte  

La règle générale est semble être une collecte des signaux 30 minutes avant la publication réelle du raport, mais c'est parfois une heure avant

Les heures théoriques de publication des résultats, utilisées pour nommer les fichiers, ne sont pas toujours vérifiées : par exemple, le classement de 2020-11-08 présenté comme étant à 14h UTC, soit 15h FR était en fait à 16h FR (d'où le dernier captage à 15h30)

    b) Création de colonnes explicites correspondant à l'analyse faite ci-dessus
- Signal request : heure de tentative de collecte du signal
- Signal reception : date et heure de réception du signal (format datetime)

Les colonnes ainsi créées doivent être capable de prendre en compte le cas où pour un classement d'un jour donné, il puisse arriver que le seul signal dont on dispose date de la veille (cas de Kevin Escoffier le 01/12/2020 à 4h)

In [11]:
# On extrait dans un df tmp les infos de la colonne heure originale
tmp=df[('Identification','Hour description')].str.extract('(\d\d:\d\d) FR(?:(-?\d\d?\d?)min)?') 
tmp=tmp.set_axis(['heure_desc','heure_offset'],axis=1)

# On convertit les deux colonnes récupérées au format datetime.time et datetime.timedelta pour pouvoir effectuer des calculs dessus
tmp['heure_desc']=pd.to_datetime(tmp['heure_desc'],format='%H:%M')
tmp['heure_offset']=tmp['heure_offset'].apply(lambda x : datetime.timedelta(minutes=int(x)) if not pd.isna(x) else datetime.timedelta(minutes=0))

# On peut ainsi obtenir l'heure des request (sans la date associée)
tmp['heure_request']=(tmp['heure_desc']-tmp['heure_offset']).dt.strftime('%H:%M')

# On construit le date-heure des requests en utiliant le fait que la request est forcément émise le même jour que la parution du classement
df[('Identification','Signal request')]=pd.to_datetime(df[('Identification','Ranking publication')].dt.strftime('%d-%m-%Y')+' '+tmp['heure_request'],format='%d-%m-%Y %H:%M')

# On construit le date-heure de réception du signal en utilisant le date-heure d'émission et l'offset
df[('Identification','Signal reception')]=df[('Identification','Signal request')]+tmp['heure_offset']

df.head(2)

Category Last signal Identification  \
Field           Rank        Sail N°   
0                  1         FRA 18   
1                  2         MON 10   

Category                                                                      \
Field                                        Skipper / Crew Hour description   
0                            Louis Burton / Bureau Vallée 2         15:30 FR   
1         Boris Herrmann / Seaexplorer - Yacht Club De M...     15:31 FR1min   

Category Last signal              Last 30min                        \
Field       Latitude   Longitude Heading (°) Speed (kts) VMG (kts)   
0         46°24.46'N  01°50.48'W        241°    17.7 kts  17.5 kts   
1         46°24.34'N  01°49.82'W        241°    11.1 kts  10.9 kts   

Category                ... Since last report    Last 24h              \
Field    Distance (nm)  ...     Distance (nm) Heading (°) Speed (kts)   
0               0.3 nm  ...         2788.0 nm        201°     0.3 kts   
1               0.4 nm  ...         2787.9 nm        196°     0.3 kts   

Category                         Last signal               Identification  \
Field    VMG (kts) Distance (nm)    DTF (nm) DTL (nm) Ranking publication   
0          0.3 kts        6.1 nm  24293.9 nm   0.0 nm 2020-11-08 14:00:00   
1          0.2 kts        6.0 nm  24294.2 nm   0.4 nm 2020-11-08 14:00:00   

Category                                          
Field         Signal request    Signal reception  
0        2020-11-08 15:30:00 2020-11-08 15:30:00  
1        2020-11-08 15:30:00 2020-11-08 15:31:00  

[2 rows x 23 columns]

    c) Vérification de la cohérence des opérations réalisées

In [12]:
# Validation du mode de calcul : on vérifie par précaution que la différence des colonnes reception et request est bien égal à l'offset extrait de la colonne initiale
((df[('Identification','Signal reception')]-df[('Identification','Signal request')]).fillna(datetime.timedelta(minutes=0))==tmp['heure_offset']).all()

True

In [13]:
# Le cas spécifique de Kevin Escoffier est bien traité correctement
df.loc[(df[('Identification','Ranking publication')].dt.date>datetime.date(2020,11,29)) & (df[('Identification','Ranking publication')].dt.date<datetime.date(2020,12,2)) & (df[('Identification',"Skipper / Crew")]=="Kevin Escoffier / PRB"),df.columns[0:3].union(df.columns[20:23])].head(10)

Category      Identification                                                  \
Field    Ranking publication Sail N°    Signal reception      Signal request   
2        2020-11-30 04:00:00  FRA 85 2020-11-30 04:30:00 2020-11-30 04:30:00   
2        2020-11-30 08:00:00  FRA 85 2020-11-30 08:30:00 2020-11-30 08:30:00   
2        2020-11-30 11:00:00  FRA 85 2020-11-30 11:30:00 2020-11-30 11:30:00   
2        2020-11-30 14:00:00  FRA 85 2020-11-30 14:30:00 2020-11-30 14:30:00   
2        2020-11-30 17:00:00  FRA 85 2020-11-30 15:00:00 2020-11-30 17:30:00   
5        2020-11-30 21:00:00  FRA 85 2020-11-30 15:00:00 2020-11-30 21:30:00   
8        2020-12-01 04:00:00  FRA 85 2020-11-30 15:00:00 2020-12-01 04:30:00   
31       2020-12-01 08:00:00  FRA 85                 NaT                 NaT   
31       2020-12-01 11:00:00  FRA 85                 NaT                 NaT   
31       2020-12-01 14:00:00  FRA 85                 NaT                 NaT   

Category                        Last signal  
Field            Skipper / Crew        Rank  
2         Kevin Escoffier / PRB           3  
2         Kevin Escoffier / PRB           3  
2         Kevin Escoffier / PRB           3  
2         Kevin Escoffier / PRB           3  
2         Kevin Escoffier / PRB           3  
5         Kevin Escoffier / PRB           6  
8         Kevin Escoffier / PRB           9  
31        Kevin Escoffier / PRB          NL  
31        Kevin Escoffier / PRB         RET  
31        Kevin Escoffier / PRB         RET

On voit précisément sur les données ci-dessus qu'à partir de 15h, aucun nouveau signal n'est reçu de la part de Kevin Escoffier. Cela correspond en fait à un naufrage, parvenu comme le laissent deviner ces données le 30 novembre 2020 à 14h46  
Lien : https://www.ouest-france.fr/vendee-globe/vendee-globe-ce-que-l-on-sait-du-naufrage-de-kevin-escoffier-7067749

In [14]:
# Vérification qu'on a bien une unique heure de request de signal pour une publication de classement donnée
((pd.crosstab(index=df[('Identification','Ranking publication')],columns=df['Identification','Signal request'])>0).sum(axis=1)==1).all()

True

La vérification ci-dessus montre qu'au moment d'élaborer un classement, le signal de collecte d'informations vers les balises des skippers est initié en même temps pour tous les concurrents

### **Réordonnancement des colonnes dans un ordre logique**

In [15]:
df=df[['Identification','Last signal','Last 30min','Since last report','Last 24h']]
df.head(2)

Category Identification                                                     \
Field           Sail N°                                     Skipper / Crew   
0                FRA 18                     Louis Burton / Bureau Vallée 2   
1                MON 10  Boris Herrmann / Seaexplorer - Yacht Club De M...   

Category                                                           \
Field    Hour description Ranking publication      Signal request   
0                15:30 FR 2020-11-08 14:00:00 2020-11-08 15:30:00   
1            15:31 FR1min 2020-11-08 14:00:00 2020-11-08 15:30:00   

Category                     Last signal                                      \
Field       Signal reception        Rank    Latitude   Longitude    DTF (nm)   
0        2020-11-08 15:30:00           1  46°24.46'N  01°50.48'W  24293.9 nm   
1        2020-11-08 15:31:00           2  46°24.34'N  01°49.82'W  24294.2 nm   

Category  ... Last 30min               Since last report              \
Field     ...  VMG (kts) Distance (nm)       Heading (°) Speed (kts)   
0         ...   17.5 kts        0.3 nm              357°     0.0 kts   
1         ...   10.9 kts        0.4 nm              357°     0.0 kts   

Category                            Last 24h                        \
Field    VMG (kts) Distance (nm) Heading (°) Speed (kts) VMG (kts)   
0          0.0 kts     2788.0 nm        201°     0.3 kts   0.3 kts   
1          0.0 kts     2787.9 nm        196°     0.3 kts   0.2 kts   

Category                
Field    Distance (nm)  
0               6.1 nm  
1               6.0 nm  

[2 rows x 23 columns]

### **Traitement des valeurs nulles**

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15873 entries, 0 to 32
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   (Identification, Sail N°)              15873 non-null  object        
 1   (Identification, Skipper / Crew)       15873 non-null  object        
 2   (Identification, Hour description)     13671 non-null  object        
 3   (Identification, Ranking publication)  15873 non-null  datetime64[ns]
 4   (Identification, Signal request)       13671 non-null  datetime64[ns]
 5   (Identification, Signal reception)     13671 non-null  datetime64[ns]
 6   (Last signal, Rank)                    15873 non-null  object        
 7   (Last signal, Latitude)                13671 non-null  object        
 8   (Last signal, Longitude)               13671 non-null  object        
 9   (Last signal, DTF (nm))                13671 non-null  object   

4 colonnes : Sail N°, Skipper / Crew, Ranking publication et Rank ne possèdent aucune valeur nulle.
Toutes les autres colonnes ont exactement le même nombre de valeurs nulles, à savoir 2202

In [17]:
pd.isna(df.loc[df[('Last signal','Rank')]=='RET']).sum()

Category           Field              
Identification     Sail N°                   0
                   Skipper / Crew            0
                   Hour description       2198
                   Ranking publication       0
                   Signal request         2198
                   Signal reception       2198
Last signal        Rank                      0
                   Latitude               2198
                   Longitude              2198
                   DTF (nm)               2198
                   DTL (nm)               2198
Last 30min         Heading (°)            2198
                   Speed (kts)            2198
                   VMG (kts)              2198
                   Distance (nm)          2198
Since last report  Heading (°)            2198
                   Speed (kts)            2198
                   VMG (kts)              2198
                   Distance (nm)          2198
Last 24h           Heading (°)            2198
                   Sp

2198 sur les 2002 valeurs nulles de chaque colonne correspondent au cas où la colonne rank vaut 'RET', c'est à dire des coureurs qui ont abandonnés. Ces informations ne sont pas utiles pour la suite donc on peut se débarasser de ces lignes

In [18]:
df=df.loc[df[('Last signal','Rank')]!='RET']
len(df)

13675

In [19]:
df.loc[df[('Last signal','Rank')]=='NL']

Category Identification                                                     \
Field           Sail N°                                     Skipper / Crew   
31               FRA 79                             Charlie Dalin / APIVIA   
32               FRA 14  Arnaud Boissieres / La Mie Câline - Artisans A...   
32               FRA 14  Arnaud Boissieres / La Mie Câline - Artisans A...   
31               FRA 85                              Kevin Escoffier / PRB   

Category                                                                       \
Field    Hour description Ranking publication Signal request Signal reception   
31                    NaN 2020-11-08 14:00:00            NaT              NaT   
32                    NaN 2020-11-08 14:00:00            NaT              NaT   
32                    NaN 2020-11-08 17:00:00            NaT              NaT   
31                    NaN 2020-12-01 08:00:00            NaT              NaT   

Category Last signal                              ... Last 30min  \
Field           Rank Latitude Longitude DTF (nm)  ...  VMG (kts)   
31                NL      NaN       NaN      NaN  ...        NaN   
32                NL      NaN       NaN      NaN  ...        NaN   
32                NL      NaN       NaN      NaN  ...        NaN   
31                NL      NaN       NaN      NaN  ...        NaN   

Category               Since last report                                      \
Field    Distance (nm)       Heading (°) Speed (kts) VMG (kts) Distance (nm)   
31                 NaN               NaN         NaN       NaN           NaN   
32                 NaN               NaN         NaN       NaN           NaN   
32                 NaN               NaN         NaN       NaN           NaN   
31                 NaN               NaN         NaN       NaN           NaN   

Category    Last 24h                                      
Field    Heading (°) Speed (kts) VMG (kts) Distance (nm)  
31               NaN         NaN       NaN           NaN  
32               NaN         NaN       NaN           NaN  
32               NaN         NaN       NaN           NaN  
31               NaN         NaN       NaN           NaN  

[4 rows x 23 columns]

Les 4 valeurs nulles restantes dans chaque colonne correspondent à un rank "NL", qui signifie probablement non localisé : il s'agit des cas où aucun signal n'a pu être remonté : 
- 3 cas sur les 4 correspondent aux deux premiers classements (probablement des skippers ayant eu des problèmes avec leur balise GPS en début de course)
- Le dernier cas correspond à l'abandon de Kevin Escoffier : il a été déclaré une fois NL avant d'être déclaré RET  

Comme précédemment, on peut se permettre de se débarasser de ces lignes

In [20]:
df=df.loc[df[('Last signal','Rank')]!='NL']
len(df)

13671

In [21]:
pd.isnull(df).any().any()

False

Il n'y a désormais plus aucune valeur nulle dans le dataframe

In [22]:
df.reset_index()

Category index Identification  \
Field                 Sail N°   
0            0         FRA 18   
1            1         MON 10   
2            2          FRA 8   
3            3         FRA 59   
4            4         FRA 53   
...        ...            ...   
13666       20         FRA 71   
13667       21         FRA 50   
13668       22         FRA 83   
13669       23         FRA 72   
13670       24        FIN 222   

Category                                                                      \
Field                                        Skipper / Crew Hour description   
0                            Louis Burton / Bureau Vallée 2         15:30 FR   
1         Boris Herrmann / Seaexplorer - Yacht Club De M...     15:31 FR1min   
2                                    Jérémie Beyou / Charal         15:30 FR   
3                                 Thomas Ruyant / LinkedOut         15:30 FR   
4                            Maxime Sorel / V And B Mayenne         15:30 FR   
...                                                     ...              ...   
13666                          Manuel Cousin / Groupe Sétin         14:30 FR   
13667                   Miranda Merron / Campagne de France         14:30 FR   
13668            Clément Giraud / Compagnie du lit - Jiliti         14:30 FR   
13669                      Alexia Barrier / TSE - 4myplanet         14:30 FR   
13670                                   Ari Huusela / Stark         14:30 FR   

Category                                                              \
Field    Ranking publication      Signal request    Signal reception   
0        2020-11-08 14:00:00 2020-11-08 15:30:00 2020-11-08 15:30:00   
1        2020-11-08 14:00:00 2020-11-08 15:30:00 2020-11-08 15:31:00   
2        2020-11-08 14:00:00 2020-11-08 15:30:00 2020-11-08 15:30:00   
3        2020-11-08 14:00:00 2020-11-08 15:30:00 2020-11-08 15:30:00   
4        2020-11-08 14:00:00 2020-11-08 15:30:00 2020-11-08 15:30:00   
...                      ...                 ...                 ...   
13666    2021-01-27 14:00:00 2021-01-27 14:30:00 2021-01-27 14:30:00   
13667    2021-01-27 14:00:00 2021-01-27 14:30:00 2021-01-27 14:30:00   
13668    2021-01-27 14:00:00 2021-01-27 14:30:00 2021-01-27 14:30:00   
13669    2021-01-27 14:00:00 2021-01-27 14:30:00 2021-01-27 14:30:00   
13670    2021-01-27 14:00:00 2021-01-27 14:30:00 2021-01-27 14:30:00   

Category Last signal                          ... Last 30min                \
Field           Rank    Latitude   Longitude  ...  VMG (kts) Distance (nm)   
0                  1  46°24.46'N  01°50.48'W  ...   17.5 kts        0.3 nm   
1                  2  46°24.34'N  01°49.82'W  ...   10.9 kts        0.4 nm   
2                  3  46°24.91'N  01°49.99'W  ...   15.5 kts        0.5 nm   
3                  4  46°24.71'N  01°49.68'W  ...   13.1 kts        0.7 nm   
4                  5  46°24.59'N  01°49.56'W  ...   10.9 kts        0.2 nm   
...              ...         ...         ...  ...        ...           ...   
13666             21  24°00.42'S  26°47.83'W  ...    7.9 kts        4.4 nm   
13667             22  26°10.76'S  26°01.91'W  ...    9.9 kts        5.4 nm   
13668             23  26°00.07'S  29°30.27'W  ...   10.9 kts        5.5 nm   
13669             24  48°09.05'S  53°51.45'W  ...   10.0 kts        5.0 nm   
13670             25  49°22.61'S  52°28.89'W  ...    9.2 kts        4.7 nm   

Category Since last report                                        Last 24h  \
Field          Heading (°) Speed (kts) VMG (kts) Distance (nm) Heading (°)   
0                     357°     0.0 kts   0.0 kts     2788.0 nm        201°   
1                     357°     0.0 kts   0.0 kts     2787.9 nm        196°   
2                     357°     0.0 kts   0.0 kts     2788.5 nm        199°   
3                     357°     0.0 kts   0.0 kts     2788.3 nm        196°   
4                     357°     0.0 kts   0.0 kts     2788.1 nm        195°   
...                    ...     

### **Conversion des colonnes numériques au format numérique**

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13671 entries, 0 to 24
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   (Identification, Sail N°)              13671 non-null  object        
 1   (Identification, Skipper / Crew)       13671 non-null  object        
 2   (Identification, Hour description)     13671 non-null  object        
 3   (Identification, Ranking publication)  13671 non-null  datetime64[ns]
 4   (Identification, Signal request)       13671 non-null  datetime64[ns]
 5   (Identification, Signal reception)     13671 non-null  datetime64[ns]
 6   (Last signal, Rank)                    13671 non-null  object        
 7   (Last signal, Latitude)                13671 non-null  object        
 8   (Last signal, Longitude)               13671 non-null  object        
 9   (Last signal, DTF (nm))                13671 non-null  object   

Les colonnes 6 à 22 doivent toutes êtres converties dans un format numérique (int ou float)

In [24]:
df[df.columns[6:]].head(2)

Category Last signal                                               Last 30min  \
Field           Rank    Latitude   Longitude    DTF (nm) DTL (nm) Heading (°)   
0                  1  46°24.46'N  01°50.48'W  24293.9 nm   0.0 nm        241°   
1                  2  46°24.34'N  01°49.82'W  24294.2 nm   0.4 nm        241°   

Category                                     Since last report              \
Field    Speed (kts) VMG (kts) Distance (nm)       Heading (°) Speed (kts)   
0           17.7 kts  17.5 kts        0.3 nm              357°     0.0 kts   
1           11.1 kts  10.9 kts        0.4 nm              357°     0.0 kts   

Category                            Last 24h                        \
Field    VMG (kts) Distance (nm) Heading (°) Speed (kts) VMG (kts)   
0          0.0 kts     2788.0 nm        201°     0.3 kts   0.3 kts   
1          0.0 kts     2787.9 nm        196°     0.3 kts   0.2 kts   

Category                
Field    Distance (nm)  
0               6.1 nm  
1               6.0 nm

In [25]:
# On commence par traiter la colonne Rank (cas le plus simple)
df[('Last signal','Rank')]=pd.to_numeric(df[('Last signal','Rank')])

In [26]:
# Pour les colonnes 9 à 22, on se débarasse des unités grâce aux regex et on convertit ensuite le résultat en valeur numérique

In [27]:
df[df.columns[9:]]=df[df.columns[9:]].applymap(lambda x : pd.to_numeric(re.match('(\d*.?\d?[^°])',x).group()))
df[df.columns[6:]].head(2)

Category Last signal                                            Last 30min  \
Field           Rank    Latitude   Longitude DTF (nm) DTL (nm) Heading (°)   
0                  1  46°24.46'N  01°50.48'W  24293.9      0.0         241   
1                  2  46°24.34'N  01°49.82'W  24294.2      0.4         241   

Category                                     Since last report              \
Field    Speed (kts) VMG (kts) Distance (nm)       Heading (°) Speed (kts)   
0               17.7      17.5           0.3               357         0.0   
1               11.1      10.9           0.4               357         0.0   

Category                            Last 24h                        \
Field    VMG (kts) Distance (nm) Heading (°) Speed (kts) VMG (kts)   
0              0.0        2788.0         201         0.3       0.3   
1              0.0        2787.9         196         0.3       0.2   

Category                
Field    Distance (nm)  
0                  6.1  
1                  6.0

In [28]:
# Reste à traiter le cas le plus complexe : la conversion des coordonnées Degrees-Minute-W/E/N/S en degrés décimaux signés
# Pour cela, j'ai créé une fonciton : 
def conver_DM_to_GPS(df,col):
    df_coord_tmp=df[('Last signal',col)].str.extract(('(\d*)°(\d*.\d*)\'(.)'))
    df_coord_tmp.set_axis(['deg','min','sign'],axis=1,inplace=True)
    df_coord_tmp['sign']=df_coord_tmp['sign'].map({'W':-1,'E':1,'S':-1,'N':1})
    df_coord_tmp=df_coord_tmp.applymap(lambda x : pd.to_numeric(x))
    return ((df_coord_tmp['deg']+df_coord_tmp['min']/60)*df_coord_tmp['sign'])

In [29]:
df[('Last signal','Longitude')]=conver_DM_to_GPS(df,'Longitude')
df[('Last signal','Latitude')]=conver_DM_to_GPS(df,'Latitude')

In [30]:
df.head(2)

Category Identification                                                     \
Field           Sail N°                                     Skipper / Crew   
0                FRA 18                     Louis Burton / Bureau Vallée 2   
1                MON 10  Boris Herrmann / Seaexplorer - Yacht Club De M...   

Category                                                           \
Field    Hour description Ranking publication      Signal request   
0                15:30 FR 2020-11-08 14:00:00 2020-11-08 15:30:00   
1            15:31 FR1min 2020-11-08 14:00:00 2020-11-08 15:30:00   

Category                     Last signal                                ...  \
Field       Signal reception        Rank   Latitude Longitude DTF (nm)  ...   
0        2020-11-08 15:30:00           1  46.407667 -1.841333  24293.9  ...   
1        2020-11-08 15:31:00           2  46.405667 -1.830333  24294.2  ...   

Category Last 30min               Since last report                        \
Field     VMG (kts) Distance (nm)       Heading (°) Speed (kts) VMG (kts)   
0              17.5           0.3               357         0.0       0.0   
1              10.9           0.4               357         0.0       0.0   

Category                  Last 24h                                      
Field    Distance (nm) Heading (°) Speed (kts) VMG (kts) Distance (nm)  
0               2788.0         201         0.3       0.3           6.1  
1               2787.9         196         0.3       0.2           6.0  

[2 rows x 23 columns]

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13671 entries, 0 to 24
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   (Identification, Sail N°)              13671 non-null  object        
 1   (Identification, Skipper / Crew)       13671 non-null  object        
 2   (Identification, Hour description)     13671 non-null  object        
 3   (Identification, Ranking publication)  13671 non-null  datetime64[ns]
 4   (Identification, Signal request)       13671 non-null  datetime64[ns]
 5   (Identification, Signal reception)     13671 non-null  datetime64[ns]
 6   (Last signal, Rank)                    13671 non-null  int64         
 7   (Last signal, Latitude)                13671 non-null  float64       
 8   (Last signal, Longitude)               13671 non-null  float64       
 9   (Last signal, DTF (nm))                13671 non-null  float64  

# **II. Caractéristiques techniques voiliers**

## **II.A. Aquisition et chargement**

### **Récupération du contenu de la page html et parsing avec BeautifulSoup**

In [32]:
r = requests.get('https://www.vendeeglobe.org/fr/glossaire')
soup=BeautifulSoup(r.content)

### **Chargement des données dans un dataframe**

On récupère d'une part le premier tag contenant les informations qui nous intéressent (descriptif du 1er bateau) et de l'autre part un générateur contenant tous les tags descriptifs des bateaux

In [33]:
ul_boat_features_1=soup.find('ul',class_='boats-list__popup-specs-list')
list_ul_boat_features=soup.find_all('ul',class_='boats-list__popup-specs-list')

On récupère les en-têtes des colonnes et on génère un dataframe vide

In [34]:
col=[]
for s in ul_boat_features_1.strings:
    if s != '\n':
        col.append(re.search('([^:]*) : .*',s.strip('\n')).group(1))
df_boats=pd.DataFrame(columns=col)
df_boats

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant


On alimente le dataframe vide avec le détail de tous les bateaux

In [35]:
n=0
for ul_boat_features in list_ul_boat_features:
    dico={}
    for s in ul_boat_features.strings:
        if s != '\n':
            dico[re.search('([^:]*) : .*',s.strip('\n')).group(1)]=re.search('[^:]*: (.*)',s.strip('\n')).group(1)
    df_boats.loc[n]=dico
    n+=1
df_boats.head(2)

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2


## **II.B. Préparation des données**

In [36]:
df_boats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 0 to 32
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Numéro de voile               32 non-null     object
 1   Anciens noms du bateau        24 non-null     object
 2   Architecte                    33 non-null     object
 3   Chantier                      33 non-null     object
 4   Date de lancement             33 non-null     object
 5   Longueur                      33 non-null     object
 6   Largeur                       33 non-null     object
 7   Tirant d'eau                  33 non-null     object
 8   Déplacement (poids)           33 non-null     object
 9   Nombre de dérives             33 non-null     object
 10  Hauteur mât                   33 non-null     object
 11  Voile quille                  32 non-null     object
 12  Surface de voiles au près     33 non-null     object
 13  Surface de voiles au p

### **Ajustement de la colonne "Numéro de voile"**

Une attention particulière doit être portée à la colonne numéro de voile car c'est celle-ci qu'on utilisera pour faire le lien avec la base de données des classements.

On constate ci-dessus qu'un bateau n'a pas de numéro de voile (NaN), Manuellement, on retrouve que c'est le bateau LinkedOut de Thomas Ruyant : on insère donc son numéro.

In [37]:
df_boats.loc[25,"Numéro de voile"]='FRA 59'

Par ailleurs, il est également nécessaire de faire correspondre les numéro de voiles des bases de données : on procède donc à un mapping via à un dictionnaire élaboré mannuellement

In [38]:
keys=[ '17','08','18','69','16','001','SUI07','4','2','FIN222','FRA09','FRA1000', 'FRA109', 'FRA30', 'FRA50', 'FRA53', 'FRA72', 'FRA83','GBR77']
values=['FRA 17','FRA 8','FRA 18','FRA 69','MON 10','FRA 01','SUI 7','FRA 4','FRA 02','FIN 222','FRA 09','FRA 1000', 'FRA 109', 'FRA 30', 'FRA 50', 'FRA 53', 'FRA 72', 'FRA 83','GBR 777']
dico=dict(zip(keys,values))
df_boats['Numéro de voile']=df_boats['Numéro de voile'].apply(lambda x : dico.get(x,x))

In [39]:
df_boats.head(2)

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2


### **Conversion des colonnes datetime et numériques dans le format python adapté**

In [40]:
df_boats.head(2)

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2


Conversion de la date au format date

In [41]:
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
df_boats['Date de lancement']=pd.to_datetime(df_boats['Date de lancement'],format ='%d %B %Y')

On répertorie l'ensemble des colonnes numériques

In [42]:
col_num=['Longueur (m)', 'Largeur (m)', 'Tirant d\'eau (m)',
       'Déplacement (t)','Hauteur mât (m)','Surface de voiles au près (m2)',
       'Surface de voiles au portant (m2)']

Intégration des unités des valeurs numériques dans le nom de la colonne

In [43]:
df_boats.set_axis(['Numéro de voile', 'Anciens noms du bateau', 'Architecte', 'Chantier',
       'Date de lancement', 'Longueur (m)', 'Largeur (m)', 'Tirant d\'eau (m)',
       'Déplacement (t)', 'Nombre de dérives', 'Hauteur mât (m)',
       'Voile quille', 'Surface de voiles au près (m2)',
       'Surface de voiles au portant (m2)'],axis=1,inplace=True)
df_boats.head(2)

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur (m),Largeur (m),Tirant d'eau (m),Déplacement (t),Nombre de dérives,Hauteur mât (m),Voile quille,Surface de voiles au près (m2),Surface de voiles au portant (m2)
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,2015-08-01,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),2007-03-08,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2


Conversion des virgules en points

In [44]:
df_boats[col_num]=df_boats[col_num].applymap(lambda x : re.sub(',','.',x))

Elimination des unités des colonnes numériques via des Regex

In [45]:
df_boats[col_num]=df_boats[col_num].applymap(lambda x : re.search('(\d*(\.\d*)?)',x).group() if not pd.isna(x) else np.nan)
df_boats.head(2)

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur (m),Largeur (m),Tirant d'eau (m),Déplacement (t),Nombre de dérives,Hauteur mât (m),Voile quille,Surface de voiles au près (m2),Surface de voiles au portant (m2)
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,2015-08-01,18.28,5.85,4.50,7,foils,29,monotype,320,570
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),2007-03-08,18.28,5.80,4.50,9,2,28,acier forgé,280,560


Conversion au format numérique

In [46]:
df_boats[col_num]=df_boats[col_num].applymap(lambda x : pd.to_numeric(x))
df_boats.head(2)

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur (m),Largeur (m),Tirant d'eau (m),Déplacement (t),Nombre de dérives,Hauteur mât (m),Voile quille,Surface de voiles au près (m2),Surface de voiles au portant (m2)
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,2015-08-01,18.28,5.85,4.5,7.0,foils,29.0,monotype,320,570
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),2007-03-08,18.28,5.80,4.5,9.0,2,28.0,acier forgé,280,560


In [47]:
df_boats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 0 to 32
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Numéro de voile                    33 non-null     object        
 1   Anciens noms du bateau             24 non-null     object        
 2   Architecte                         33 non-null     object        
 3   Chantier                           33 non-null     object        
 4   Date de lancement                  33 non-null     datetime64[ns]
 5   Longueur (m)                       33 non-null     float64       
 6   Largeur (m)                        33 non-null     float64       
 7   Tirant d'eau (m)                   33 non-null     float64       
 8   Déplacement (t)                    31 non-null     float64       
 9   Nombre de dérives                  33 non-null     object        
 10  Hauteur mât (m)                    33 no